In [19]:
import os
import re
import requests
from lxml import etree
from selenium import webdriver
import pandas as pd
import shutil

In [20]:
headers = [{
    "name":
    "Accept",
    "value":
    "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8"
}, {
    "name": "Accept-Encoding",
    "value": "gzip, deflate, br"
}, {
    "name":
    "Accept-Language",
    "value":
    "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2"
}, {
    "name": "Connection",
    "value": "keep-alive"
}, {
    "name": "Cookie",
    "value": "existmag=mag; PHPSESSID=k4karuqn837cuk0ntdq0g4mor1"
}, {
    "name": "Host",
    "value": "www.javbus.com"
}, {
    "name": "Sec-Fetch-Dest",
    "value": "document"
}, {
    "name": "Sec-Fetch-Mode",
    "value": "navigate"
}, {
    "name": "Sec-Fetch-Site",
    "value": "none"
}, {
    "name": "Sec-Fetch-User",
    "value": "?1"
}, {
    "name": "TE",
    "value": "trailers"
}, {
    "name": "Upgrade-Insecure-Requests",
    "value": "1"
}, {
    "name":
    "User-Agent",
    "value":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0"
}]
headers = {i['name']:i["value"] for i in headers}

In [21]:
df = pd.read_csv("./250.csv",names=["NO", "code"])
df.head(5)

,NO,code
0,1,LAFBD-41
1,2,ABP-984
2,3,SSNI-497
3,4,IPX-580
4,5,IPX-811


In [21]:
executable_path = r"E:\JupyterFiles\geckodriver\geckodriver\geckodriver.exe"
# driver = webdriver.Firefox(executable_path=executable_path)

driver = webdriver.Firefox()

In [19]:
def get_magnet_urls(code):
    query_url = "https://javdb.com/search?q={}&f=all".format(code)
    # print(query_url)
    driver.get(query_url)
    html = etree.HTML(driver.page_source)
    href_xpath = './/div[@class="item"][1]/a/@href'
    href = ["https://javdb.com" + i for i in html.xpath(href_xpath)][-1]
    # print(href)
    
    driver.get(href)
    html = etree.HTML(driver.page_source)
    a =  html.xpath('.//div[@class="magnet-name column is-four-fifths"]/a')
    magnet_urls = html.xpath('.//div[@class="magnet-name column is-four-fifths"]/a/@href')
    #print(url)
    if html.xpath('.//span[text()="字幕"]/../..'):
        max_size = html.xpath('.//span[text()="字幕"]/../../span[@class="meta"]/text()')[0]
        max_size = float(re.findall("\d+.\d+", max_size)[0])
        
        for _i,size in enumerate(html.xpath('.//span[text()="字幕"]/../../span[@class="meta"]/text()')):
            size = float(re.findall("\d+.\d+", size)[0])
            
            # print(max_size, size)
            
            if size > max_size:
                # print(max_size, _i)
                magnet_url = html.xpath('.//span[text()="字幕"]/../..')[_i].attrib.get("href")
            else:
                # print("diyi")
                magnet_url = html.xpath('.//span[text()="字幕"]/../..')[0].attrib.get("href")
            max_size = size if size > max_size else max_size
        
        
        # print("有字幕")
    elif html.xpath('.//span[text()="高清"]/../..'):
        magnet_url = html.xpath('.//span[text()="高清"]/../..')[0].attrib.get("href")
        # print("HD")
    else:
        magnet_url = magnet_urls[0]
    return magnet_url 

In [4]:
fhs = [re.findall("\w+-(\d+)", ele)[-1] for ele in os.listdir(r"G:\JAV\TOP")]
fhs = [i.upper() for i in fhs]

In [5]:
fhs

['001',
 '017',
 '171',
 '408',
 '889',
 '912',
 '933',
 '968',
 '975',
 '984',
 '995',
 '032',
 '054',
 '087',
 '108',
 '121',
 '135',
 '153',
 '187',
 '204',
 '216',
 '220',
 '244',
 '286',
 '296',
 '305',
 '324',
 '339',
 '396',
 '344',
 '557',
 '243',
 '259',
 '365',
 '404',
 '120',
 '778',
 '092',
 '789',
 '077',
 '272',
 '513',
 '529',
 '548',
 '504',
 '0783',
 '1031',
 '033',
 '765',
 '047',
 '050',
 '118',
 '177',
 '216',
 '247',
 '398',
 '451',
 '471',
 '475',
 '484',
 '497',
 '515',
 '528',
 '534',
 '536',
 '540',
 '557',
 '565',
 '580',
 '582',
 '596',
 '602',
 '607',
 '620',
 '633',
 '641',
 '643',
 '660',
 '666',
 '690',
 '697',
 '708',
 '726',
 '743',
 '776',
 '777',
 '811',
 '832',
 '850',
 '851',
 '867',
 '917',
 '002',
 '003',
 '034',
 '037',
 '179',
 '185',
 '388',
 '887',
 '048',
 '136',
 '722',
 '41',
 '234',
 '568',
 '605',
 '698',
 '831',
 '525',
 '548',
 '715',
 '797',
 '020',
 '633',
 '767',
 '776',
 '906',
 '934',
 '960',
 '970',
 '979',
 '036',
 '207',
 '229',

In [23]:
for i in df["code"][::1]:
    
    try:
        fh = re.findall("\w+-(\d+)", i)[-1]
    except:
#         print(i)
        pass
    if fh not in fhs:
#         print(i)
        print(get_magnet_urls(i), "\n")

magnet:?xt=urn:btih:448dc38fa2d50b0314999940c57bf88891db5f92&dn=[javdb.com]abw-096-C.torrent 

magnet:?xt=urn:btih:e524db8fbba7a401fe53718e4a3420cb87ecf368&dn=[javdb.com]121914-760-C.torrent 

magnet:?xt=urn:btih:r762a3lacipgpdnh43dtzxoaetaywmrn&dn=[javdb.com]IPX-868-C_X1080X 

magnet:?xt=urn:btih:60aa09e36b56550d1d4beeeb75d2ef7a5654544a&dn=[javdb.com]ipx-416-C.torrent 

magnet:?xt=urn:btih:956343e0cff87b9c7357d5e667e12bc81469017e&dn=[javdb.com]ssni-618-C.torrent 

magnet:?xt=urn:btih:335165b666e6b45e57697bb26294f6ff07c75392&dn=[javdb.com]MKBD-S03-BD.torrent 

magnet:?xt=urn:btih:2ff0ac27cf0189f75372ccc85200f0849318ebdb&dn=[javdb.com]pred-262-C.torrent 

magnet:?xt=urn:btih:2465e92dcb72bbdaed856356fe2e5502d4333258&dn=[javdb.com]ABW-366-UC.torrent.无码破解 

magnet:?xt=urn:btih:9ac2ca710577cfc209e6cca82be9481d000f729e&dn=[javdb.com]IPZ-127.1080p.x264.AAC.中文特效字幕.mp4 

magnet:?xt=urn:btih:8d83621ba82dce44fbb30ab7b02a7bbe3e4ecace&dn=[javdb.com]ABW-043.torrent.无码破解 

magnet:?xt=urn:btih:8931742

In [24]:
for i in df["code"][::1]:
    
    try:
        fh = re.findall("\w+-(\d+)", i)[-1]
    except:
        print(i)

010115_001
MKBD-S03
MKBD-S89
032715_002
vixen.16.10.06
092415_159
blackedraw.19.05.17


In [12]:
executable_path = r"E:\JupyterFiles\geckodriver\geckodriver\geckodriver.exe"
# driver = webdriver.Firefox(executable_path=executable_path)

driver = webdriver.Firefox()

In [2]:
def get_href(fh:str):
    """
    根据番号获取跳转链接
    """
    # 查询链接
    seacher_url = "https://javdb.com/search?q={}&f=all".format(fh)
    # print(seacher_url)
    res_text = driver.get(seacher_url)
    
    html = etree.HTML(driver.page_source)
    img_src = html.xpath('.//div[@class="movie-list h cols-4 vcols-8"]/div[@class="item"]/a/@href')[0]
    img_src = "https://javdb.com" + img_src
    return img_src

In [153]:
root = r"G:\JAV\TOP\75-101"
for i in os.listdir(root)[::-1]:
    src_folder = "{}\{}".format(root, i)
    fh = ("").join(i.split("-")[:2:])
    
    print(get_href(fh))
    
    driver.get(get_href(fh))
    html = etree.HTML(driver.page_source)
    try:
        NO_text = html.xpath('.//span[text()="JavDB 影片TOP250"]/preceding-sibling::span/text()')[0]
    except:
        shutil.move(src_folder, r"G:\JAV\TOP\年度\{}".format(i))
        pass
    NO = re.findall(pattern="\d+", string=NO_text)[0]
    if int(NO) < 51:
        shutil.move(src_folder, r"G:\JAV\TOP\1-50\{}".format(i))
    elif int(NO) > 50 and int(NO) < 61:
        shutil.move(src_folder, r"G:\JAV\TOP\51-60\{}".format(i))
    elif int(NO) > 60 and int(NO) < 91:
        shutil.move(src_folder, r"G:\JAV\TOP\60-90\{}".format(i))
    elif int(NO) > 90 and int(NO) < 161:
        shutil.move(src_folder, r"G:\JAV\TOP\90-160\{}".format(i))
    elif int(NO) > 180 and int(NO) < 260:
        shutil.move(src_folder, r"G:\JAV\TOP\180-250\{}".format(i))

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'G:\\JAV\\TOP\\75-101'

In [18]:
root = r"G:\JAV\TOP\83-86"
for i in os.listdir(root)[::-1]:
    src_folder = "{}\{}".format(root, i)
    fh = ("").join(i.split("-")[:2:])
    
    print(get_href(fh))
    
    driver.get(get_href(fh))
    html = etree.HTML(driver.page_source)
    try:
        NO_text = html.xpath('.//span[text()="JavDB 影片TOP250"]/preceding-sibling::span/text()')[0]
    except:
        shutil.move(src_folder, r"G:\JAV\TOP\年度\{}".format(i))
        pass
    NO = re.findall(pattern="\d+", string=NO_text)[0]
    print(NO)
    if int(NO) < 0:
        shutil.move(src_folder, r"G:\JAV\TOP\151-158\{}".format(i))
    elif int(NO) > 86 and int(NO) < 95:
        shutil.move(src_folder, r"G:\JAV\TOP\87-95\{}".format(i))
    elif int(NO) > 90 and int(NO) < 96:
        shutil.move(src_folder, r"G:\JAV\TOP\91-95\{}".format(i))
    else:
        pass
    

https://javdb.com/v/PrYe2
90
https://javdb.com/v/WzXz8
83
https://javdb.com/v/K2Xmv
84
https://javdb.com/v/wGDbB
86
https://javdb.com/v/5EWRKD
89
https://javdb.com/v/YeK7p
85
https://javdb.com/v/PnBkN
87


In [23]:
try:
    driver.get("https://www.javbus.com")
except:
    driver = webdriver.Firefox()

root = r"G:\JAV\top250"
for i in os.listdir(root)[::-1]:
    src_folder = "{}\{}".format(root, i)
    fh = ("").join(i.split("-")[:2:])
    
    print(get_href(fh))
    
    driver.get(get_href(fh))
    html = etree.HTML(driver.page_source)
    try:
        NO_text = html.xpath('.//span[text()="JavDB 影片TOP250"]/preceding-sibling::span/text()')[0]
    except:
        shutil.move(src_folder, r"G:\JAV\TOP\年度\{}".format(i))
        pass
    NO = re.findall(pattern="\d+", string=NO_text)[0]
    print(NO)
driver.close()

https://javdb.com/v/dazNv
87
https://javdb.com/v/W1M4dK
206
https://javdb.com/v/B84DpG
215
